**Load all libraries**

In [ ]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
# import plotly.express as px
import openpyxl
import requests
import datetime
import re
# Main source https://www.gov.uk/government/statistics/company-insolvencies-mmmm-yyyy

In [ ]:
# Get most recent months data
current_month = datetime.datetime.now().strftime('%B-%Y').lower()
prior_month = (datetime.datetime.now() - datetime.timedelta(days=30)).strftime('%B-%Y').lower()
print(f'current_month: {current_month}')
print(f'prior_month: {prior_month}')

In [ ]:
url_current = f'https://www.gov.uk/government/statistics/company-insolvencies-{current_month}'
url_prior = f'https://www.gov.uk/government/statistics/company-insolvencies-{prior_month}'
response = None

try:
    os.makedirs('content')
except:
    pass

Main Extraction

In [ ]:
# First, attempt to get the current month's URL.
print(f"Attempting to fetch current month URL: {url_current}")
try:
    response = requests.get(url_current)
    response.raise_for_status()  # This will raise an exception for 404, 500, etc.
    print('Current Month data found. Extracting...')

except requests.exceptions.RequestException as e:
    # If the current month request fails (e.g., a 404 Not Found error),
    # this block handles the fallback logic.
    print(f"Current month URL failed with an error: {e}")
    print(f"Attempting to fetch prior month URL: {url_prior}")

    try:
        response = requests.get(url_prior)
        response.raise_for_status()
        print("Prior month data found. Extracting...")
    except requests.exceptions.RequestException as e:
        # If the prior month request also fails, this block executes.
        print(f"Prior month URL also failed with an error: {e}")
        print("Could not retrieve data from either URL. Aborting.")
        response = None  # Ensure response is explicitly None

# --- Main Logic: Only run if a valid response was received ---
# This conditional check is the key to preventing the crash.
if response:
    # Use regular expressions to find the Excel link in the HTML content.
    # This is more robust than looking for a hard-coded string.
    # We are looking for any link that ends with '.xlsx'.
    excel_link = None
    # Compile a regular expression to find an href ending in .xlsx
    pattern = re.compile(r'href="([^"]+\.xlsx)"')

    # Search for the pattern in the response text
    match = pattern.search(response.text)
    if match:
        excel_link = match.group(1)

    if excel_link:
        # Construct the full URL if the link is relative.
        if not excel_link.startswith('http'):
            # The 'url' property of the response object gives us the base URL.
            excel_link = requests.compat.urljoin(response.url, excel_link)

        # Use the name of the file from the link itself for a dynamic file name.
        file_name = excel_link.split('/')[-1]

        # Download the Excel file
        try:
            excel_response = requests.get(excel_link)
            excel_response.raise_for_status()

            # Save the file. Use 'wb' for write binary mode.
            with open(f'content/{file_name}', 'wb') as f:
                f.write(excel_response.content)

            print(f"Downloaded '{file_name}' to /content/")
        except requests.exceptions.RequestException as e:
            print(f"Failed to download the Excel file: {e}")
    else:
        print("Excel file link not found on the page.")

else:
    print("Skipping data processing as no valid response was retrieved.")


In [ ]:
# file_name = 'Data_Tables_in_Excel__xlsx__Format_-_Company_Insolvency_Statistics_July_2025.xlsx' # placeholder for offline testing

Load required dataset


In [ ]:
# print(f'{file_name}')
try:
  df = pd.read_excel(f'content/{file_name}')
  print('Data loaded')
  df.head(1)
except FileNotFoundError:
  print('File not found')
  exit()

In [ ]:
sheet_to_extract = 'Table_1c'
df_single_sheet = pd.read_excel((f'content/{file_name}'), sheet_name=sheet_to_extract, skiprows=7)
df_single_sheet.head()

In [ ]:
header_row_index = 0
description_column_index = 0
is_all_caps = ~df_single_sheet['Description'].astype(str).str.contains('[a-z]', na=False)
is_not_total = ~df_single_sheet['Description'].astype(str).str.contains('[-]', na=False)

df_remove_lower_case = df_single_sheet[is_all_caps].copy()
df_filtered = df_remove_lower_case[is_not_total].copy()

df_filtered.drop(['Section','Division','Group','Notes'], axis=1, inplace=True)
df_filtered.head()

In [ ]:
start_col_loc = df_filtered.columns.get_loc('Jan 2022')

# Find the last column that does NOT contain '[x]'
end_col_loc = start_col_loc
# Iterate backwards from the last column to the start column
for i in range(len(df_filtered.columns) - 1, start_col_loc - 1, -1):
    col_name = df_filtered.columns[i]
    # Check if any cell in the column contains the '[x]' marker
    if not df_filtered[col_name].astype(str).str.contains(r'\[x\]', regex=True).any():
        # If no '[x]' is found, this is our last valid column
        end_col_loc = i
        break
# Select the final columns
# Create a list of all column indices to keep:
# The description column + the range of valid data columns
cols_to_keep_indices = [description_column_index] + list(range(start_col_loc, end_col_loc + 1))
df_relevant_columns = df_filtered.iloc[:, cols_to_keep_indices]
df_relevant_columns.head()

In [ ]:
print("Formatting and cleaning data...")
# Pivot to required format
df_formatted = pd.melt(df_relevant_columns, id_vars=['Description'], var_name='Date', value_name='Value')
# Clean columns
df_formatted['Value'] = df_formatted['Value'].astype(str).str.replace(',', '', regex=False)
df_formatted['Value'] = pd.to_numeric(df_formatted['Value'], errors='coerce')
# Remove NA values
df_formatted.dropna(subset=['Value'], inplace=True)

# Change Description column to Category
df_formatted['Date'] = pd.to_datetime(df_formatted['Date'], format='%b %Y')
df_formatted.rename(columns={'Description': 'Category'}, inplace=True)

# Rename a long category name
long_name = 'ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS-AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE'
short_name = 'UNDIFFERENTIATED G&S PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE'
df_formatted['Category'] = df_formatted['Category'].replace(long_name, short_name)

df_formatted.head()

In [ ]:
pivot_df = df_formatted.pivot_table(
    index=df_formatted['Date'].dt.to_period('M'),
    columns='Category',
    values='Value',
    aggfunc='sum',
    fill_value=0
)
pivot_df.head()

In [ ]:
df_formatted['Year'] = df_formatted['Date'].dt.year

yearly_pivot_df = df_formatted.pivot_table(
    index='Year',
    columns='Category',
    values='Value',
    aggfunc='sum',
    fill_value=0
)
yearly_pivot_df.head()

# **Experimental Section**

In [ ]:
# @title Annual Grouping

# Create the stacked bar chart from the pivoted data
# ax = pivot_df.plot(kind='bar', stacked=True, figsize=(12, 8), width=0.8)
ax = yearly_pivot_df.plot(kind='bar', stacked=True, figsize=(12, 8), width=0.8)

# Customize the plot for better readability
ax.set_title('Month-to-Month Values by Category', fontsize=16)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Value', fontsize=12)
plt.xticks(rotation=90, ha='right')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')

# Display the chart
plt.show()

In [ ]:
# @title Monthly Grouping
ax_bar = pivot_df.plot(
    kind='bar',
    stacked=True,
    figsize=(14, 8),
    width=0.8  # Makes bars wider and reduces the gap. Default is 0.5.
)

ax_bar.set_title('Monthly Values by Category (Stacked Bar)', fontsize=16)
ax_bar.set_xlabel('Month', fontsize=8)
ax_bar.set_ylabel('Total Companies', fontsize=8)
plt.xticks(rotation=90, ha='right')
plt.legend(title='Category', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# @title Categrories
# ----> IMPORTANT: Choose which categories you want to plot here <----
# You can see all available categories by uncommenting the line below:
import pprint
# print(f"Available categories: {pivot_df.columns.tolist()}")
pprint.pp(pivot_df.columns.tolist())

In [ ]:
# @title Line graph of selected categories
selected_categories = pivot_df.columns.tolist()

selected_categories = [
    'ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES',
    # 'CONSTRUCTION',
    # 'WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
    # 'CONSTRUCTION' # Add or remove categories from this list
]

# Filter the DataFrame to only include the columns (categories) you selected
# This check ensures that the categories exist before trying to plot them
# valid_categories = [cat for cat in selected_categories if cat in pivot_df.columns]
valid_categories = [cat for cat in selected_categories if cat in pivot_df.columns]

if not valid_categories:
    print("Warning: None of the selected categories were found in the data. Skipping line graph.")
else:
    ax_line = pivot_df[valid_categories].plot(
        kind='line',
        figsize=(14, 8),
        marker='o'  # Adds circles to data points for better visibility
    )

    ax_line.set_title('Monthly Values for Selected Categories (Line)', fontsize=16)
    ax_line.set_xlabel('Month', fontsize=12)
    ax_line.set_ylabel('Value', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.legend(title='Selected Categories')
    plt.tight_layout()
    plt.show()


In [ ]:
# @title 3 Month Moving Average Graph

import pandas as pd
import matplotlib.pyplot as plt

monthly_totals = df_formatted.groupby(df_formatted['Date'].dt.to_period('M'))['Value'].sum()

WINDOW_SIZE = 3

# Check if there is enough data
if len(monthly_totals) < WINDOW_SIZE:
    print(f"**Warning:** You have only {len(monthly_totals)} months of data. "
          f"A {WINDOW_SIZE}-month moving average requires at least {WINDOW_SIZE} months to produce a single value and more show a trend.")

moving_average = monthly_totals.rolling(window=WINDOW_SIZE).mean()

# Prepare data with string index for stable plotting
plot_df = pd.DataFrame({
    'Total Value': monthly_totals,
    'Moving Average': moving_average
})
# Set the index to your desired string format ('Jan 2022')
plot_df.index = plot_df.index.strftime('%b %Y')


# Step 3: Plot the DataFrame with the corrected string index
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(figsize=(14, 7))

plot_df['Total Value'].plot(ax=ax, label='Total Monthly Value', marker='.', linestyle='-')
plot_df['Moving Average'].plot(ax=ax, label=f'{WINDOW_SIZE}-Month Moving Average', linestyle='--', linewidth=2)

# Formatting the plot
ax.set_title('Total Value and Moving Average Over Time', fontsize=18, pad=20)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Total Value', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotate the labels
ax.legend(fontsize=12)
fig.tight_layout()
ax.set_ylim(bottom=0)

plt.show()


In [ ]:
# @title Category Changes over time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Assume 'df_formatted' is your DataFrame

print("\n--- Generating Category Growth Analysis ---")

pivot_df = df_formatted.pivot_table(
    index=df_formatted['Date'].dt.to_period('M'),
    columns='Category',
    values='Value',
    aggfunc='sum',
    fill_value=0
)

slopes = {}
x_axis = np.arange(len(pivot_df.index))
for category in pivot_df.columns:
    slope, _ = np.polyfit(x_axis, pivot_df[category], 1)
    slopes[category] = slope

slopes_series = pd.Series(slopes)
highest_growth_cat = slopes_series.idxmax()
lowest_growth_cat = slopes_series.idxmin()

print(f"Category with HIGHEST growth trend: '{highest_growth_cat}'")
print(f"Category with LOWEST growth trend: '{lowest_growth_cat}'")

# --- FIX: Prepare data with string index for stable plotting ---
plot_pivot_df = pivot_df.copy()
plot_pivot_df.index = plot_pivot_df.index.strftime('%b %Y')


# Step 3: Plot the DataFrame with the corrected string index
fig, ax = plt.subplots(figsize=(14, 7))

plot_pivot_df[highest_growth_cat].plot(ax=ax, label=f'Highest Trend: {highest_growth_cat}', marker='o', linestyle=':')
plot_pivot_df[lowest_growth_cat].plot(ax=ax, label=f'Lowest Trend: {lowest_growth_cat}', marker='o', linestyle=':')
plot_pivot_df['WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES'].plot(ax=ax, label=f'Investigating Trend: WHOLESALE AND RETAIL TRADE', marker='o', linestyle=':')
# plot_pivot_df['ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES'].plot(ax=ax, label=f'Investigating Trend: WHOLESALE AND RETAIL TRADE', marker='o', linestyle=':')

# Formatting the plot
ax.set_title('Category trends with number of insolvencies', fontsize=18, pad=20)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Number of Company insolvencies', fontsize=12)
plt.xticks(rotation=45, ha='right') # Rotate the labels
ax.legend(fontsize=12)
ax.set_ylim(bottom=0)
fig.tight_layout()
plt.show()


In [ ]:
# @title Volatility by industry
volatility = (pivot_df.std() / pivot_df.mean()) * 100

print("\n--- Volatility Score (Coefficient of Variation %) ---")
# A higher score means more volatility and less predictability
print(volatility.sort_values(ascending=False))

In [ ]:
slopes_series

In [ ]:

# @title Year-to-Date (YTD) Analysis

# Find the latest month in your data
# latest_month = df_formatted['Date'].dt.month.max()
latest_year = df_formatted['Date'].dt.year.max()
latest_month = df_formatted[df_formatted['Date'].dt.year == latest_year]['Date'].dt.month.max()


print(f"--- Comparing Year-to-Date performance (up to month {latest_month}) ---")

# Filter the DataFrame to only include data up to that month for all years
ytd_df = df_formatted[df_formatted['Date'].dt.month <= latest_month].copy()

# Create a 'Year' column
ytd_df['Year'] = ytd_df['Date'].dt.year

# Group by year and sum the values for the YTD period
ytd_totals = ytd_df.groupby('Year')['Value'].sum()

# --- Plotting the YTD Comparison ---
fig, ax = plt.subplots(figsize=(10, 6))
ytd_totals.plot(
    kind='bar',
    ax=ax,
    color='skyblue',
    edgecolor='black'
)

ax.set_title(f'Year-to-Date Performance (Months 1-{latest_month})', fontsize=16)
ax.set_xlabel('Year', fontsize=12)
ax.set_ylabel('Total YTD Value', fontsize=12)
plt.xticks(rotation=0)
ax.set_ylim(bottom=0)
fig.tight_layout()
plt.show()


In [ ]:
# @title Single category vs peers

CHOSEN_CATEGORY = 'WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES'

# How many other stable categories you want to compare against
NUM_SIMILAR_CATEGORIES = 3
# The window for the moving average
WINDOW_SIZE = 3


# Find the Most Stable Categories
# First, we need to pivot the data to analyze each category's time series
pivot_df = df_formatted.pivot_table(
    index=df_formatted['Date'].dt.to_period('M'),
    columns='Category',
    values='Value',
    aggfunc='sum',
    fill_value=0
)

# Calculate volatility (a lower score is more stable)
volatility = (pivot_df.std() / pivot_df.mean()) * 100
# Sort by volatility to get a ranked list of the most stable categories
stable_categories = volatility.sort_values(ascending=True)


#Select Categories and Calculate Moving Averages
if CHOSEN_CATEGORY not in pivot_df.columns:
    print(f"Error: Category '{CHOSEN_CATEGORY}' was not found in the data.")
else:
    # Get the top N most stable categories, excluding the one we already chose
    similar_cats_to_plot = stable_categories[stable_categories.index != CHOSEN_CATEGORY].head(NUM_SIMILAR_CATEGORIES).index.tolist()

    # Calculate the moving average for all categories
    moving_averages_df = pivot_df.rolling(window=WINDOW_SIZE).mean()

    # Prepare a DataFrame for plotting with a clean string index
    plot_df = moving_averages_df.copy()
    plot_df.index = plot_df.index.strftime('%b %Y')

    # --- 4. Plotting the Comparison ---
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax = plt.subplots(figsize=(14, 8))

    # Plot the CHOSEN_CATEGORY with a distinct, solid line
    plot_df[CHOSEN_CATEGORY].plot(
        ax=ax,
        label=f'Main Focus: {CHOSEN_CATEGORY}',
        linewidth=3,
        color='black',
        marker='.'
    )

    # Plot the other stable categories with lighter, dashed lines
    for cat in similar_cats_to_plot:
        plot_df[cat].plot(
            ax=ax,
            label=f'Stable Peer: {cat}',
            linestyle='--',
            linewidth=1.5
        )

    # Formatting the plot
    ax.set_title(f'Trend Comparison: Wholesale and retail trade vs. Stable Peers', fontsize=18, pad=20)
    ax.set_xlabel('Month', fontsize=12)
    ax.set_ylabel(f'{WINDOW_SIZE}-Month Moving Average Value', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    ax.legend(title='Category Comparison')
    ax.set_ylim(bottom=0)
    fig.tight_layout()
    ax.legend(
        title='Category Comparison',
        frameon=True,
        facecolor='white',
        edgecolor='black',
        shadow=True,
        fancybox=True
   )
    plt.show()